<a href="https://colab.research.google.com/github/FatemaTabassum/Data_mining_ML_Revise/blob/main/Advance_data_mining_naive_bayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import re
from collections import defaultdict
import sys
import collections
import math


In [2]:
import re
from collections import defaultdict
import sys
import collections
import math

class NaiveBayesClassifier:
  def __init__(self) -> None:
    self.prior_prob_dict = {}
    self.keep_count_dict = {}
    self.class_1_cnt = 0
    self.class_2_cnt = 0
    self.class_1_identifier = 0
    self.class_2_identifier = 1
    self.dataset_train = []
    self.labels_train = []
    self.dataset_test = []
    self.labels_test = []
    self.max_attribute = 0
    self.attribute_value_counts = defaultdict(set) # To store unique values for each attribute

  #r red from files
  def read_files(self, train_filename, test_filename):
    with open(train_filename, 'r') as f:
      train_set = f.readlines()
    with open(test_filename, 'r') as f:
      test_set = f.readlines()
    return train_set, test_set

  # create a list of dataset
  def create_dataset(self, raw_data_set):
    labels = []
    dataset = []
    max_id = 0
    for line in raw_data_set:
      res_val = re.split(r'[ ]', line.strip()) # Use strip() to remove leading/trailing whitespace
      ind = 0
      attribute_dict = {} # Use a dictionary to store attributes
      for val in res_val:
        if ind == 0:
          if val == '+1':
            labels.append(self.class_1_identifier)      # +1 is categorized as class 0
          else:
            labels.append(self.class_2_identifier)      # -1 is categorized as class 1
        else:
          if ':' in val: # Ensure the value contains ':' before splitting
            v = re.split(r'\:', val)
            id = int(v[0])          # index starts from 1
            num = int(v[1])
            max_id = max(max_id, id)
            attribute_dict[id] = num # Store attribute in dictionary
            self.attribute_value_counts[id].add(num) # Track unique values for smoothing
        # incrementing index for attribute
        ind +=1
      dataset.append(attribute_dict)
    self.max_attribute = max_id # Update the max_attribute instance variable
    return labels, dataset


  def compute_prior_prob(self, labels):
    self.class_1_cnt = labels.count(self.class_1_identifier)
    self.class_2_cnt = labels.count(self.class_2_identifier)
    total_instances = len(labels)
    if total_instances > 0:
        self.prior_prob_dict[self.class_1_identifier] = self.class_1_cnt / total_instances
        self.prior_prob_dict[self.class_2_identifier] = self.class_2_cnt / total_instances
    else:
        # Handle case with no data to avoid division by zero
        self.prior_prob_dict[self.class_1_identifier] = 0
        self.prior_prob_dict[self.class_2_identifier] = 0
    return self.prior_prob_dict

  def classifier_train(self, dataset, labels):
    # Initialize keep_count_dict as a defaultdict to handle missing keys easily
    self.keep_count_dict = defaultdict(lambda: defaultdict(int)) # Nested defaultdict for class -> attribute_id -> value -> count

    for i in range(0, len(dataset)):
        class_label = labels[i]
        attributes = dataset[i]
        for attr_id, value in attributes.items():
            self.keep_count_dict[class_label][attr_id][value] += 1

    #print(self.keep_count_dict) # Optional: for debugging

  def compute_likelyhood(self, dataset_):
      computed_labels = []
      for j in range(0, len(dataset_)):
        class_1_log_likelihood_sum = 0 # Use log probabilities and sum them
        class_2_log_likelihood_sum = 0 # Use log probabilities and sum them

        attributes = dataset_[j] # Get attributes for the current instance

        # Iterate through all possible attributes up to self.max_attribute
        # This is important for smoothing, even if an attribute is not present in the current instance
        # in the test set, we need to consider its probability based on the training data.
        # We will iterate through all unique attribute IDs seen during training.
        all_attribute_ids = set(self.attribute_value_counts.keys()) | set(attributes.keys())


        for attr_id in all_attribute_ids:
          value = attributes.get(attr_id, 0) # Get the value if present, default to 0 if not

          # Determine the number of possible values for this attribute based on training data
          num_possible_values = len(self.attribute_value_counts[attr_id]) if attr_id in self.attribute_value_counts else 2 # Assume binary if not seen

          # check with class 1
          # Use the nested defaultdict structure for lookup
          count_class1 = self.keep_count_dict.get(self.class_1_identifier, {}).get(attr_id, {}).get(value, 0)

          # Add 1 smoothing (Laplace smoothing) to avoid zero probabilities
          smoothed_count_class1 = count_class1 + 1
          # Calculate denominator for smoothing: total count of instances in class 1 + number of possible values for this attribute
          smoothing_denominator_class1 = self.class_1_cnt + num_possible_values

          if smoothing_denominator_class1 > 0:
              prob_class1 = smoothed_count_class1 / smoothing_denominator_class1
              if prob_class1 > 0: # Avoid log(0)
                class_1_log_likelihood_sum += math.log(prob_class1)
              else:
                  # Handle the case where probability is still zero after smoothing (shouldn't happen with +1 smoothing and num_possible_values >= 1)
                  # but as a safeguard, add a very small value or handle appropriately.
                  # For simplicity, we'll just skip adding to the sum if prob is 0 after smoothing.
                  pass


          # check with class 2
          count_class2 = self.keep_count_dict.get(self.class_2_identifier, {}).get(attr_id, {}).get(value, 0)

          smoothed_count_class2 = count_class2 + 1
          smoothing_denominator_class2 = self.class_2_cnt + num_possible_values

          if smoothing_denominator_class2 > 0:
              prob_class2 = smoothed_count_class2 / smoothing_denominator_class2
              if prob_class2 > 0: # Avoid log(0)
                class_2_log_likelihood_sum += math.log(prob_class2)
              else:
                  # Handle the case where probability is still zero after smoothing
                  pass


        # Adding log of prior probabilities (correctly by adding logarithms)
        if self.class_1_identifier in self.prior_prob_dict and self.prior_prob_dict[self.class_1_identifier] > 0:
             class_1_log_likelihood_sum += math.log(self.prior_prob_dict[self.class_1_identifier])

        if self.class_2_identifier in self.prior_prob_dict and self.prior_prob_dict[self.class_2_identifier] > 0:
            class_2_log_likelihood_sum += math.log(self.prior_prob_dict[self.class_2_identifier])


        if class_1_log_likelihood_sum >= class_2_log_likelihood_sum:
          computed_labels.append(self.class_1_identifier)
        else:
          computed_labels.append(self.class_2_identifier)

      return computed_labels

In [ ]:
# Assuming the NaiveBayesClassifier class is defined in a previous cell
# Let's create a sample raw dataset string
sample_raw_data = """+1 1:1 5:1 10:1
-1 2:1 5:0 8:1
+1 1:0 5:1 12:0"""

# Split the sample data into lines
sample_raw_data_set = sample_raw_data.strip().split('\n')

# Instantiate the NaiveBayesClassifier
nb_classifier = NaiveBayesClassifier()

# Call the create_dataset method with the sample data
max_id, labels, dataset = nb_classifier.create_dataset(sample_raw_data_set)

# Print the results
print(f"max_id: {max_id}")
print(f"labels: {labels}")
print("Sample dataset entries:")
for i in range(min(3, len(dataset))): # Print up to 3 sample entries
  print(f"  {dataset[i]}")

max_id: 12
labels: [0, 1, 0]
Sample dataset entries:
  [0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
  [0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
  [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [ ]:
# This creates a list with 20 zeros
attribute_ar = [0] * 20

# This prints the list
print(attribute_ar)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
